# GAN sur MNIST (TensorFlow) - Version 2 avec MLflow

In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Reshape, LeakyReLU
from tensorflow.keras.models import Sequential
import numpy as np
import matplotlib.pyplot as plt
import mlflow
import mlflow.tensorflow

# Initialisation MLflow
mlflow.set_experiment("MNIST_GAN_TF")


ModuleNotFoundError: No module named 'mlflow'

In [ ]:
# Charger les données MNIST
(x_train, _), (_, _) = tf.keras.datasets.mnist.load_data()
x_train = x_train.astype("float32") / 255.0
x_train = x_train.reshape(-1, 28*28)

BUFFER_SIZE = 60000
BATCH_SIZE = 128
LATENT_DIM = 100
EPOCHS = 50

train_dataset = tf.data.Dataset.from_tensor_slices(x_train).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)


In [ ]:
# Générateur
def build_generator():
    model = Sequential([
        Dense(128, input_shape=(LATENT_DIM,), activation=LeakyReLU(0.2)),
        Dense(784, activation='sigmoid'),
        Reshape((28, 28))
    ])
    return model

# Discriminateur
def build_discriminator():
    model = Sequential([
        Flatten(input_shape=(28, 28)),
        Dense(128, activation=LeakyReLU(0.2)),
        Dense(1, activation='sigmoid')
    ])
    return model


In [ ]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=False)

generator = build_generator()
discriminator = build_discriminator()

generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)


In [ ]:
@tf.function
def train_step(images):
    noise = tf.random.normal([BATCH_SIZE, LATENT_DIM])

    with tf.GradientTape() as disc_tape, tf.GradientTape() as gen_tape:
        generated_images = generator(noise, training=True)

        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        disc_loss = cross_entropy(tf.ones_like(real_output), real_output) + \
                    cross_entropy(tf.zeros_like(fake_output), fake_output)
        gen_loss = cross_entropy(tf.ones_like(fake_output), fake_output)

    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)

    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))
    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))

    return gen_loss, disc_loss


In [ ]:
gen_losses = []
disc_losses = []

def train(dataset, epochs):
    with mlflow.start_run():
        mlflow.log_param("batch_size", BATCH_SIZE)
        mlflow.log_param("latent_dim", LATENT_DIM)
        mlflow.log_param("epochs", epochs)

        for epoch in range(epochs):
            epoch_gen_loss = 0
            epoch_disc_loss = 0
            steps = 0

            for image_batch in dataset:
                gen_loss, disc_loss = train_step(image_batch)
                epoch_gen_loss += gen_loss
                epoch_disc_loss += disc_loss
                steps += 1

            avg_gen_loss = epoch_gen_loss / steps
            avg_disc_loss = epoch_disc_loss / steps

            gen_losses.append(avg_gen_loss.numpy())
            disc_losses.append(avg_disc_loss.numpy())

            mlflow.log_metric("generator_loss", avg_gen_loss.numpy(), step=epoch)
            mlflow.log_metric("discriminator_loss", avg_disc_loss.numpy(), step=epoch)

            if epoch % 10 == 0:
                print(f"Epoch {epoch}, Generator Loss: {avg_gen_loss:.4f}, Discriminator Loss: {avg_disc_loss:.4f}")


In [ ]:
train(train_dataset, EPOCHS)

In [ ]:
# Courbes de perte
plt.figure(figsize=(10, 5))
plt.plot(gen_losses, label="Generator Loss", linewidth=2)
plt.plot(disc_losses, label="Discriminator Loss", linewidth=2)
plt.title("Courbes d'apprentissage GAN (MNIST)")
plt.xlabel("Époch")
plt.ylabel("Perte")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()
